# <center> Text Generation - Next Word Prediction

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/nlp-specialization-data/'):
    print(dirname)
    #for filename in filenames:
    #   print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-specialization-data/
/kaggle/input/nlp-specialization-data/Medical_Notes
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes
/kaggle/input/nlp-specialization-data/pubmed2018_w2v_200D
/kaggle/input/nlp-specialization-data/pubmed2018_w2v_200D/pubmed2018_w2v_200D


## 1. Functions for Processing Text

### a. Reading in files as a string text

In [74]:
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [3]:
len("sakshi")

5

In [4]:
len(read_file('/kaggle/input/nlp-specialization-data/Novel - Moby-Dick By Herman Melville.txt'))

1198622

In [5]:
print(read_file('/kaggle/input/nlp-specialization-data/Novel - Moby-Dick By Herman Melville.txt')[:5000])

CHAPTER 1

Loomings.


Call me Ishmael.  Some years ago--never mind how long
precisely--having little or no money in my purse, and nothing
particular to interest me on shore, I thought I would sail about a
little and see the watery part of the world.  It is a way I have of
driving off the spleen and regulating the circulation.  Whenever I
find myself growing grim about the mouth; whenever it is a damp,
drizzly November in my soul; whenever I find myself involuntarily
pausing before coffin warehouses, and bringing up the rear of every
funeral I meet; and especially whenever my hypos get such an upper
hand of me, that it requires a strong moral principle to prevent me
from deliberately stepping into the street, and methodically knocking
people's hats off--then, I account it high time to get to sea as soon
as I can.  This is my substitute for pistol and ball.  With a
philosophical flourish Cato throws himself upon his sword; I quietly
take to the ship.  There is nothing surprising in this

### b. Tokenize and Clean Text

In [6]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()
nlp.max_length = 1198623

In [7]:
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text[:250000]

In [8]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [9]:
d = read_file('/kaggle/input/nlp-specialization-data/Novel - Moby-Dick By Herman Melville.txt')
tokens = separate_punc(d)

In [10]:
type(tokens)

list

In [11]:
len(tokens)

45279

In [12]:
tokens[:10]

['chapter',
 '1',
 'loomings',
 'call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never']

### c. Create Sequences of Tokens

In [13]:
# organize into sequences of tokens
train_len = 25+1 # 25 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [14]:
print(read_file('/kaggle/input/nlp-specialization-data/Novel - Moby-Dick By Herman Melville.txt')[:1000])

CHAPTER 1

Loomings.


Call me Ishmael.  Some years ago--never mind how long
precisely--having little or no money in my purse, and nothing
particular to interest me on shore, I thought I would sail about a
little and see the watery part of the world.  It is a way I have of
driving off the spleen and regulating the circulation.  Whenever I
find myself growing grim about the mouth; whenever it is a damp,
drizzly November in my soul; whenever I find myself involuntarily
pausing before coffin warehouses, and bringing up the rear of every
funeral I meet; and especially whenever my hypos get such an upper
hand of me, that it requires a strong moral principle to prevent me
from deliberately stepping into the street, and methodically knocking
people's hats off--then, I account it high time to get to sea as soon
as I can.  This is my substitute for pistol and ball.  With a
philosophical flourish Cato throws himself upon his sword; I quietly
take to the ship.  There is nothing surprising in this

In [15]:
' '.join(text_sequences[0])

'chapter 1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to'

In [16]:
' '.join(text_sequences[1])

'1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest'

In [17]:
' '.join(text_sequences[2])

'loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me'

In [18]:
len(text_sequences) #Every sentence is containing 26 words

45253

In [19]:
len(tokens) # These are total number of words in the whole novel

45279

In [20]:
print(len(text_sequences[0]))
print(text_sequences[0])

26
['chapter', '1', 'loomings', 'call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to']


### d. Keras Tokenization

In [21]:
from keras.preprocessing.text import Tokenizer

In [22]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [23]:
print(len(sequences[0]))
print(sequences[0])

26
[187, 7000, 6999, 516, 24, 432, 65, 232, 640, 100, 223, 99, 93, 1331, 277, 68, 48, 36, 639, 6, 31, 1028, 2, 150, 431, 5]


In [24]:
type(tokenizer.index_word)

dict

In [25]:
' '.join(text_sequences[0])

'chapter 1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to'

In [26]:
i=0
for a in tokenizer.index_word:
    print(a,"--->",tokenizer.index_word[a])
    i+=1
    if i==20 : break 

1 ---> the
2 ---> and
3 ---> of
4 ---> a
5 ---> to
6 ---> in
7 ---> i
8 ---> that
9 ---> his
10 ---> he
11 ---> it
12 ---> was
13 ---> but
14 ---> with
15 ---> all
16 ---> for
17 ---> as
18 ---> 's
19 ---> at
20 ---> is


In [27]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

187 : chapter
7000 : 1
6999 : loomings
516 : call
24 : me
432 : ishmael
65 : some
232 : years
640 : ago
100 : never
223 : mind
99 : how
93 : long
1331 : precisely
277 : having
68 : little
48 : or
36 : no
639 : money
6 : in
31 : my
1028 : purse
2 : and
150 : nothing
431 : particular
5 : to


In [28]:
i=0
for a in tokenizer.word_counts:
    print((a,tokenizer.word_counts[a]))
    i+=1
    if i==10 : break 

('chapter', 807)
('1', 2)
('loomings', 3)
('call', 238)
('me', 6214)
('ishmael', 292)
('some', 2451)
('years', 606)
('ago', 191)
('never', 1700)


In [29]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

7000

In [30]:
i=6990
for a in range(i,7000):
    print(a,"--->",tokenizer.index_word[a])
    #i+=1
    #if i==6999 : break 

6990 ---> subtle
6991 ---> conceived
6992 ---> poorly
6993 ---> moods
6994 ---> whimsicalities
6995 ---> apprehensiveness
6996 ---> uneasiness
6997 ---> warrantry
6998 ---> emotions
6999 ---> loomings


In [31]:
# tokenizer.index_word

### e. Convert to Numpy Matrix

In [32]:
import numpy as np

In [33]:
len(sequences)

45253

In [34]:
len(sequences[0])

26

In [35]:
sequences = np.array(sequences)

In [36]:
sequences.shape

(45253, 26)

In [37]:
sequences

array([[ 187, 7000, 6999, ...,  150,  431,    5],
       [7000, 6999,  516, ...,  431,    5,  734],
       [6999,  516,   24, ...,    5,  734,   24],
       ...,
       [  86,  578,    7, ...,  472,   14,    1],
       [ 578,    7,  171, ...,   14,    1,  111],
       [   7,  171,    5, ...,    1,  111,  577]])

## 2. Creating an LSTM based model

In [38]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [39]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

## 3. Train / Test Split

In [40]:
from tensorflow.keras.utils import to_categorical

In [41]:
print(sequences.shape)
sequences

(45253, 26)


array([[ 187, 7000, 6999, ...,  150,  431,    5],
       [7000, 6999,  516, ...,  431,    5,  734],
       [6999,  516,   24, ...,    5,  734,   24],
       ...,
       [  86,  578,    7, ...,  472,   14,    1],
       [ 578,    7,  171, ...,   14,    1,  111],
       [   7,  171,    5, ...,    1,  111,  577]])

In [44]:
# First 25 words
print(sequences[:,:-1].shape)
sequences[:,:-1]

(45253, 25)


array([[ 187, 7000, 6999, ...,    2,  150,  431],
       [7000, 6999,  516, ...,  150,  431,    5],
       [6999,  516,   24, ...,  431,    5,  734],
       ...,
       [  86,  578,    7, ...,    1,  472,   14],
       [ 578,    7,  171, ...,  472,   14,    1],
       [   7,  171,    5, ...,   14,    1,  111]])

In [43]:
# last Word
print(sequences[:,-1].shape)
sequences[:,-1]

(45253,)


array([  5, 734,  24, ...,   1, 111, 577])

In [45]:
X = sequences[:,:-1]

In [46]:
X.shape

(45253, 25)

In [47]:
y = sequences[:,-1]

In [48]:
y.shape

(45253,)

In [49]:
y = to_categorical(y, num_classes=vocabulary_size)

In [50]:
y.shape

(45253, 7000)

In [51]:
seq_len = X.shape[1]

In [52]:
seq_len

25

## 4. Training the Model

In [53]:
# define model
model = create_model(vocabulary_size, seq_len)
#model = create_model(vocabulary_size, seq_len)

2021-11-20 16:24:57.041712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 16:24:57.042838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 16:24:57.043516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 16:24:57.044607: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            175000    
_________________________________________________________________
lstm (LSTM)                  (None, 25, 150)           105600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense (Dense)                (None, 150)               22650     
_________________________________________________________________
dense_1 (Dense)              (None, 7000)              1057000   
Total params: 1,540,850
Trainable params: 1,540,850
Non-trainable params: 0
_________________________________________________________________


---

----

## 5. Fit model

In [54]:
model.fit(X, y, batch_size=512, epochs=250,verbose=1,validation_batch_size=.20)

2021-11-20 16:26:25.305955: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1267084000 exceeds 10% of free system memory.
2021-11-20 16:26:26.883605: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1267084000 exceeds 10% of free system memory.
2021-11-20 16:26:27.939913: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/250


2021-11-20 16:26:31.202639: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


89/89 [==============================] - 7s 24ms/step - loss: 7.2391 - accuracy: 0.0538
Epoch 2/250
89/89 [==============================] - 2s 24ms/step - loss: 6.7368 - accuracy: 0.0546
Epoch 3/250
89/89 [==============================] - 2s 25ms/step - loss: 6.7163 - accuracy: 0.0546
Epoch 4/250
89/89 [==============================] - 2s 24ms/step - loss: 6.6181 - accuracy: 0.0546
Epoch 5/250
89/89 [==============================] - 2s 24ms/step - loss: 6.5013 - accuracy: 0.0554
Epoch 6/250
89/89 [==============================] - 2s 24ms/step - loss: 6.3722 - accuracy: 0.0619
Epoch 7/250
89/89 [==============================] - 2s 24ms/step - loss: 6.2853 - accuracy: 0.0691
Epoch 8/250
89/89 [==============================] - 2s 25ms/step - loss: 6.2179 - accuracy: 0.0714
Epoch 9/250
89/89 [==============================] - 2s 26ms/step - loss: 6.1610 - accuracy: 0.0744
Epoch 10/250
89/89 [==============================] - 2s 24ms/step - loss: 6.1139 - accuracy: 0.0760
Epoch 11/25

### Download Model Object

In [56]:
from pickle import dump,load

In [57]:
# save the model to file
model.save('epochBIG.h5')
# save the tokenizer
dump(tokenizer, open('epochBIG', 'wb'))

## 6. Generating New Text

In [58]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [59]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (25 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        #pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        predict_x=model.predict(pad_encoded) 
        pred_word_ind=np.argmax(predict_x,axis=1)[0]
        #print(pred_word_ind)
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

## 7. Grab a random Text Sequence

In [61]:
print(text_sequences[0])

['chapter', '1', 'loomings', 'call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to']


In [62]:
import random
random_pick = random.randint(0,len(text_sequences))

In [63]:
random_seed_text = text_sequences[random_pick]

In [65]:
print(random_seed_text)

['events', 'soberly', 'recurred', 'one', 'by', 'one', 'in', 'fixed', 'reality', 'and', 'then', 'i', 'lay', 'only', 'alive', 'to', 'the', 'comical', 'predicament', 'for', 'though', 'i', 'tried', 'to', 'move', 'his']


In [66]:
seed_text = ' '.join(random_seed_text)
seed_text

'events soberly recurred one by one in fixed reality and then i lay only alive to the comical predicament for though i tried to move his'

In [67]:
model = load_model('epochBIG.h5')

In [68]:
tokenizer = load(open('epochBIG', 'rb'))

In [69]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=3)

'arm unlock his'

In [70]:
text=' It is a way I have of driving off the spleen and regulating the circulation.  Whenever I find'
print(text)

 It is a way I have of driving off the spleen and regulating the circulation.  Whenever I find


In [71]:
generate_text(model,tokenizer,seq_len,seed_text=text,num_gen_words=3)

'myself growing grim'

In [75]:
print(read_file('/kaggle/input/nlp-specialization-data/Novel - Moby-Dick By Herman Melville.txt')[250000:251000])

f the crew, were a far more barbaric, heathenish, and
motley set than any of the tame merchant-ship companies which my
previous experiences had made me acquainted with, still I ascribed
this--and rightly ascribed it--to the fierce uniqueness of the very
nature of that wild Scandinavian vocation in which I had so
abandonedly embarked.  But it was especially the aspect of the three
chief officers of the ship, the mates, which was most forcibly
calculated to allay these colourless misgivings, and induce confidence
and cheerfulness in every presentment of the voyage.  Three better,
more likely sea-officers and men, each in his own different way,
could not readily be found, and they were every one of them
Americans; a Nantucketer, a Vineyarder, a Cape man.  Now, it being
Christmas when the ship shot from out her harbor, for a space we had
biting Polar weather, though all the time running away from it to the
southward; and by every degree and minute of latitude which we
sailed, gradually lea

In [81]:
test_text='Three better,more likely sea-officers and men, each in his own different way,could not readily be found, and they were every'
print(test_text)

Three better,more likely sea-officers and men, each in his own different way,could not readily be found, and they were every


In [83]:
generate_text(model,tokenizer,seq_len,seed_text=test_text,num_gen_words=1)

'god'

In [ ]:
seed_text = "landsman has had fresh fruit to his daily hand and broken the world 's fresh bread to my mouldy crusts away whole oceans away from that"

In [ ]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=3)